Importing all Libraries needed to train as well as get information from the data base

In [124]:
import pandas as pd
import logging
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pytds


Connecting to database to retrieve student and training resumes

In [125]:
connect_string = ""
try:
    con = pytds.connect(server = '10.236.2.124', database = 'TBDraft', user = 'admin1', password='TBuilder')
    cursor = con.cursor()
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

INFO - 09:15:45: Opening socket to 10.236.2.124:1433
INFO - 09:15:45: Sending PRELOGIN lib_ver=10b0000 enc_flag=2 inst_name=b'MSSQLServer' mars=False
INFO - 09:15:45: Got PRELOGIN response crypt=2 mars=0
INFO - 09:15:45: Sending LOGIN tds_ver=74000004 bufsz=4096 pid=18276 opt1=f0 opt2=2 opt3=8 cli_tz=-360 cli_lcid=1033 cli_host=DESKTOP-02AVBJA lang= db=TBDraft
INFO - 09:15:45: switched to database TBDraft
INFO - 09:15:45: switched collation to Collation(lcid=1033, sort_id=52, ignore_case=True, ignore_accent=False, ignore_width=True, ignore_kana=True, binary=False, binary2=False, version=0)
INFO - 09:15:45: switched language to us_english
INFO - 09:15:45: Got LOGINACK tds_ver=74000004 srv_name=Microsoft SQL Server   srv_ver=f0007d0
INFO - 09:15:45: Sending BEGIN TRAN il=0


 is working


In [126]:
##After connecting to database extract preprocessed information
#These are the base training resumes
df_training = pd.read_sql_query("SELECT ALL * FROM TrainingResumes", con)
#There are the processed student resumes
df_students = pd.read_sql_query("SELECT ALL * FROM StudentInformation", con)
con.close()
display(df_training)
display(df_students)


INFO - 09:15:45: Sending query SELECT ALL * FROM TrainingResumes
INFO - 09:15:45: Sending query SELECT ALL * FROM StudentInformation


,keyword,dataText
0,Biomedical_Engineer,daniel whatcott service manager biomedical e...
1,Biomedical_Engineer,patrick hannah electrical engineer ii varex ...
2,Electronics_Engineer,adam blakeslee north ogden ut email work exp...
3,Electronics_Engineer,alaa issa electrical engineering iraq sulaym...
4,Electronics_Engineer,juan avila charleston sc email experience el...
...,...,...
144,Software_Engineer,nathan nguyen irvine email work experience s...
145,Software_Engineer,omar trueba mijango computer engineer san jo...
146,Software_Engineer,peter pagano bohemia ny email authorize work...
147,Software_Engineer,steven nichol c c software engineer grand ra...


,ID,StudentName,GPA,Email,StudentResume,ProcessedData,PhoneNumber,StringResume
0,1,Andrew Esfandiari,None,None,b'%PDF-1.7\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,andrew esfandiari federal way wa email autho...,None,None
1,2,Arturo Cervantes,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,arturo cervante mission tx email seek employ...,None,None
2,3,Bobby Crenshaw,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,bobby crenshaw lead ad hoc trial aim windstr...,None,None
3,4,Brandon Karnoski,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,brandon karnoski league city tx email work e...,None,None
4,5,Damien Jeppson,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,damien jeppson college station tx email seni...,None,None
5,6,daniyal khaleel,None,None,None,daniyal khaleel lahore va email software eng...,None,None
6,7,Daod Mohammad,None,None,None,daod mohammad rolling hill estate email prof...,None,None
7,8,Frank Cisneros,None,None,None,frank cisneros embed software engineer richa...,None,None
8,9,Gabriel Kollat,None,None,None,gabriel kollat justin tx email authorize wor...,None,None
9,10,Jess Stacy,None,None,None,jess stacy passionate computer engineer prog...,None,None


In [127]:
df = pd.DataFrame(columns=['Keyword', 'Data'])
df['Keyword'] = df_training['keyword'].append(df_students['ID'], ignore_index=True)
df['Data'] = df_training['dataText'].append(df_students['ProcessedData'], ignore_index=True)

For Now we use tika to parse resumes and clean them

In [128]:
# for i in range(len(df['Data'])):
#     sent.append((df.Data[i].split()))
sent = [row.split() for row in df['Data']] #Change change ['clean'] to whatever we actually label the columns as
documents = [TaggedDocument(doc, tags=[df.Keyword[i]]) for i, doc in enumerate(sent)]

Training the model and saving it

In [129]:
threads = multiprocessing.cpu_count()
model = Doc2Vec(documents, 
                dm = 0,
                dbow_words=1, 
                alpha=0.03, 
                vector_size=30, 
                min_alpha = 0.0007, 
                min_count = 10, 
                negative=20,
                workers=threads-1)
model.save('d2v_model')

INFO - 09:15:46: collecting all words and their counts
INFO - 09:15:46: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 09:15:46: collected 9021 word types and 30 unique tags from a corpus of 169 examples and 85737 words
INFO - 09:15:46: Creating a fresh vocabulary
INFO - 09:15:46: Doc2Vec lifecycle event {'msg': 'effective_min_count=10 retains 1453 unique words (16.10686176698814%% of original 9021, drops 7568)', 'datetime': '2021-11-30T09:15:46.769390', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
INFO - 09:15:46: Doc2Vec lifecycle event {'msg': 'effective_min_count=10 leaves 68934 word corpus (80.40169355120894%% of original 85737, drops 16803)', 'datetime': '2021-11-30T09:15:46.770982', 'gensim': '4.0.1', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 

In [130]:
tokens ="nlp machine learn analytic".split()
new_vec = model.infer_vector(tokens)
sims = model.dv.most_similar(positive=[new_vec])
display(sims)
model.similarity_unseen_docs(doc_words1=df.Data[161].split(), doc_words2="nlp machine learn analytic".split())

[(13, 0.8560515642166138),
 ('Machine_Learning', 0.7658829689025879),
 (20, 0.7612653970718384),
 (12, 0.6976027488708496),
 (10, 0.6113545298576355),
 (8, 0.5897161960601807),
 (4, 0.5661092400550842),
 (15, 0.5414218902587891),
 (6, 0.5346149802207947),
 ('Embedded_Systems', 0.5304047465324402)]

0.8169346

In [131]:
con.close()

In [132]:
model.dv.index_to_key

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 'Biomedical_Engineer',
 'Electronics_Engineer',
 'Power_Systems_Engineer',
 'Software_Engineer',
 'Embedded_Systems',
 'Electronic_Project_Management',
 'Machine_Learning',
 'RF_Analog_Engineer',
 'Renewable_Energy']